In [2]:
import os, time, random, logging , datetime , cv2 , csv , subprocess
import numpy as np


import tensorflow as tf
print("tf",tf.version.VERSION)
from tensorflow import keras

from utils import globo , xdv , tfh5 , sinet


''' GPU CONFIGURATION '''

#os.environ["CUDA_VISIBLE_DEVICES"]="1,2"
#tfh5.set_tf_loglevel(logging.ERROR)
#tfh5.tf.debugging.set_log_device_placement(False) #Enabling device placement logging causes any Tensor allocations or operations to be printed.
#tfh5.set_memory_growth()
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


CFG_SINET = {
    
    'sinet_version': 'fsd-sinet-vgg42-tlpf_aps-1',
    
    'graph_filename' : os.path.join(globo.FSDSINET_PATH,"fsd-sinet-vgg42-tlpf_aps-1.pb"),
    'metadata_file'  : os.path.join(globo.FSDSINET_PATH,"fsd-sinet-vgg42-tlpf_aps-1.json"),
    
    'audio_fs_input':22050,
    'batchSize' : 64,
    'lastPatchMode': 'repeat',
    'patchHopSize' : 50,
    
    
    'anom_labels' : ["Alarm","Boom","Crowd","Dog","Drill","Explosion","Fire","Gunshot and gunfire","Hammer","Screaming","Screech",\
                    "Shatter","Shout","Siren","Slam","Squeak","Yell"],
    'anom_labels_i' : [4,18,51,59,65,72,78,92,94,145,146,147,148,152,154,161,198],
    
    'anom_labels2' : ["Boom","Explosion","Fire","Gunshot and gunfire","Screaming",\
                    "Shout","Siren","Yell"],
    'anom_labels_i2' : [18,72,78,92,147,148,152,198],
    
    'full_or_max' : 'max', #chose output to (timesteps,labels_total) ot (1,labels_total)
    'labels_total' : 200
    
}
CFG_WAV= {
    
    "full_or_max" : CFG_SINET["full_or_max"],
    "sinet_aas_len" : CFG_SINET["labels_total"], 
    
    "shuffle" : False,
    
    "ativa" : 'relu',
    "optima" : 'sgd',
    "batch_type" : 0,   # =0 all batch have frame_max or video length // =1 last batch has frame_max frames // =2 last batch has no repetead frames
    "frame_max" : 8000,
    "ckpt_start" : f"{0:0>8}",  #used in train_model: if 00000000 start from scratch, else start from ckpt with CFG_WAV stated
    
    "epochs" : 1,
    "batch_size" : 1
    
}

tf 2.2.0


2023-05-03 02:59:24.431071: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-03 02:59:24.431143: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2023-05-03 02:59:25.597167: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 02:59:25.605126: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-03 02:59:26.786889

In [3]:
video = cv2.VideoCapture('/raid/DATASETS/anomaly/XD_Violence/testing_copy/v=38GQ9L2meyE__#1_label_B6-0-0.mp4');tot_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT));fps=int(video.get(cv2.CAP_PROP_FPS));video.release()

In [4]:
#xdv.create_train_valdt_test_from_xdvtest_bg()
data=xdv.train_valdt_test_from_xdvtest_bg_from_npy(True)



***** train ******************


1 ANOMALY
   724 total intervals
   311277 total frames
   3.6027430555555555 total hours

0 NORMAL
   785 total intervals
   319403 total frames
   3.696793981481482 total hours

Interval counts: {1: 37, 2: 43, 3: 70, 4: 29, 5: 32, 6: 18, 7: 22, 8: 5, 9: 7, 10: 7, 11: 6, 12: 9, 13: 3, 14: 5, 15: 1, 17: 1, 19: 1, 20: 1, 32: 1, 35: 1, 44: 1}

***** valdt ******************


1 ANOMALY
   280 total intervals
   131413 total frames
   1.5209837962962964 total hours

0 NORMAL
   300 total intervals
   127530 total frames
   1.4760416666666667 total hours

Interval counts: {1: 11, 2: 17, 3: 20, 4: 6, 5: 14, 6: 3, 7: 7, 9: 2, 11: 2, 12: 6, 13: 2, 14: 2, 15: 3, 16: 2, 17: 1, 20: 1, 34: 1}

***** test ******************


1 ANOMALY
   234 total intervals
   95634 total frames
   1.106875 total hours

0 NORMAL
   251 total intervals
   83848 total frames
   0.970462962962963 total hours

Interval counts: {1: 11, 2: 23, 3: 21, 4: 7, 5: 9, 6: 5, 7: 6, 8: 2, 9: 3,

In [ ]:
sinet = sinet.Sinet(CFG_SINET)

In [ ]:
len(data['train'][7][1])

In [ ]:
def create_npz_pes_frame_interval():
    
    data_dicts= {'train': [], 'valdt': [], 'test': []}

    for i in range(len(data)): 
                
        if not i:typee = "train"
        elif i == 1:typee = "valdt"
        else: typee = "test"
        print("\n*****",typee,"******************\n")
        
        total_intervals = 0
        for video_j in range(len(data[typee])):
            line = data[typee][video_j]
            vpath = data[typee][video_j][0]
            frame_intervals = data[typee][video_j][1] #(sf,ef,label)
            
            total_intervals += len(frame_intervals)
            
            p_es_array = sinet.get_sigmoid_fl(vpath,frame_intervals)
            
            # Store the vpath, frame_interval, p_es_array, and label for each interval
            for k in range(len(p_es_array)):
                data_dicts[typee].append({
                    'vpath': vpath,
                    'frame_interval': frame_intervals[k],
                    'p_es_array': p_es_array[k],
                    'label': frame_intervals[k][2]
                })
                print(vpath,frame_intervals[k],np.shape(p_es_array[k]),frame_intervals[k][2])
            
            #print("\n\n\n")
            #for k in range(len(p_es_array)):
            #    print("interval",k,"@",np.shape(p_es_array[k]),frame_intervals[k][2])
            
        print("\n\n\n***********************\n",total_intervals,len(data_dicts[typee]))
            
    # Save the data for each typee to a single .npz file
    for typee in data_dicts:
        ofn = f"p_es-frame_intrv-{typee}.npz"
        ofp = os.path.join(globo.SERVER_TEST_COPY_PATH+'npy',ofn)
        print(ofn,'\n',ofp)
        # Save the vpath, frame_interval, p_es_array, and label as .npz files
        np.savez_compressed(ofp, data=data_dicts[typee])

In [ ]:
fp = '/raid/DATASETS/anomaly/XD_Violence/annotations.txt'
data = []
with open(fp, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        video_path = os.path.join(globo.SERVER_TEST_COPY_PATH,str(parts[0])+'.mp4')
        video = cv2.VideoCapture(video_path);tot_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT));fps=int(video.get(cv2.CAP_PROP_FPS));video.release()
        intervals = [(int(parts[i]), int(parts[i + 1])) for i in range(1, len(parts), 2)]
        print(intervals)
        
        ## Check for a relevant normal interval at the beginning of the video
        if int(intervals[0][0]) > fps * 2:
            first_interval = (0, int(intervals[0][0]) - 1)
            new_intervals = [first_interval] + intervals
            
        else: new_intervals = intervals
            
        ## Check for a relevant normal interval at the end of the video
        if tot_frames - int(intervals[-1][-1]) > fps * 2:
            data.append((video_path, new_intervals, tot_frames))
            
        else: data.append((video_path, new_intervals))
        
        print(data[-1])